<a href="https://colab.research.google.com/github/Sumitkevlani/LLM/blob/main/Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bidict                           0.23.1

In [20]:
pip install langchain python-dotenv openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [13]:
import langchain
import os

**Function Calling in OpenAI**

* Output can be formatted in the desired format.
* Output of any third party API is appended into the Conversation Chain.

If the LLM model is not able to answer something then the output of the third party API can be appended into the conversation chain.





In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
KEY = os.getenv("OPENAI_API_KEY")


In [17]:
student_description = "Sumit Kevlani is a 19 year-old and passionate Programmer currently pursuing his B.tech in Information Technology from NIT, Jalandhar. His grade is 8.68"

In [18]:
prompt = f'''
Please extract the following information from the given text and return it as a JSON object:
name
college
branch
cgpa
The text is: {student_description}
'''

In [19]:
prompt

'\nPlease extract the following information from the given text and return it as a JSON object:\nname\ncollege\nbranch\ncgpa\nThe text is: Sumit Kevlani is a 19 year-old and passionate Programmer currently pursuing his B.tech in Information Technology from NIT, Jalandhar. His grade is 8.68\n'

In [21]:
from openai import OpenAI
client = OpenAI(api_key=KEY)

The below method is used for chat completions. Here, we passed the model that is the gpt-3.5-turbo model.

Here, messages is the list of dictionary having two things: role and content.

So, role are of three types: user, assistant and system.

user role is for the client or the user using the model.

assistant role is for the AI model or the model responding to the queries.

system role is for the background context to set the tone of the conversation.

Examples:
- user sets the prompt
- assistant gives the AI message response
- system sets the background context(You are an helpful assistant.)

Note:
- The prompt here is an example of few shot prompt as we have given background context to the model and said it to behave like this.
- If we directly ask some question from the model then that prompt is known as zero shot prompt.



In [23]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role":"user",
            "content":prompt
        }
    ]
)

In [24]:
response

ChatCompletion(id='chatcmpl-9SjSodwoulKowcKMptqPEZUzR05Ok', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "name": "Sumit Kevlani",\n  "college": "NIT, Jalandhar",\n  "branch": "Information Technology",\n  "cgpa": 8.68\n}', role='assistant', function_call=None, tool_calls=None))], created=1716634974, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=44, prompt_tokens=77, total_tokens=121))

In [26]:
output = response.choices[0].message.content

In [27]:
import json
json.loads(output)

{'name': 'Sumit Kevlani',
 'college': 'NIT, Jalandhar',
 'branch': 'Information Technology',
 'cgpa': 8.68}

In [30]:
functions = [
    {
        "name": "get_student_details",
        "description": "Get the details of the Student from the text.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the Student."
                },
                "college": {
                    "type": "string",
                    "description": "The institution in which the student studies."
                },
                "branch": {
                    "type": "string",
                    "description": "The branch of the student"
                },
                "cgpa": {
                    "type": "string",
                    "description": "The grade point of the student."
                }
            },
            "required": ["name","college","branch","cgpa"]
        }
    }
]

In [31]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role":"user",
            "content":prompt
        }
    ],
    functions = functions
)
print(response)

ChatCompletion(id='chatcmpl-9SkC2wMFGuS9B2GOeCM82NUPUITHj', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"name":"Sumit Kevlani","college":"NIT, Jalandhar","branch":"Information Technology","cgpa":"8.68"}', name='get_student_details'), tool_calls=None))], created=1716637778, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=41, prompt_tokens=164, total_tokens=205))


In [33]:
print(response.choices[0].message.function_call.arguments)

{"name":"Sumit Kevlani","college":"NIT, Jalandhar","branch":"Information Technology","cgpa":"8.68"}


**Advanced use case of function calling**

If a question is unanswered by OpenAI then we can call the respective API to get the answer. So, here the GPT model was not able to answer the question but using OpenAI function calling we extracted the arguments which the user entered and found the complete information for the same.

In [37]:
functions = [
    {
        "name":"get_flight_info",
        "description": "Get flight information from origin to destination",
        "parameters": {
            "type":"object",
            "properties": {
                "loc_origin": {
                    "type":"string",
                    "description": "Get origin location"
                },
                "loc_destination": {
                    "type": "string",
                    "description": "Get destination location"
                },
                "flight_no": {
                    "type": "string",
                    "description": "The flight number of the flight"
                },
                "itinerary_time": {
                    "type": "string",
                    "description": "The total time of the trip"
                }
            },
            "required": ["loc_origin","loc_destination"]
        }
    }
]

In [38]:
prompt = "What is the next flight from Jodhpur to Pune?"

In [39]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role":"user",
            "content":prompt
        }
    ],
    functions = functions
)
print(response)

ChatCompletion(id='chatcmpl-9SklsSrlbYoaCnYjHPJE7yYouH3jQ', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"loc_origin":"Jodhpur","loc_destination":"Pune"}', name='get_flight_info'), tool_calls=None))], created=1716640000, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=25, prompt_tokens=104, total_tokens=129))


In [42]:
flight_response = response.choices[0].message.function_call.arguments

In [44]:
flight_response = json.loads(flight_response)
print(flight_response)

{'loc_origin': 'Jodhpur', 'loc_destination': 'Pune'}


In [59]:
from datetime import datetime,timedelta
def get_flight_info(loc_origin,loc_destination):
  flight_info = {
      "loc_origin": loc_origin,
      "loc_destination": loc_destination,
      "flight_no": "A320",
      "itinerary_time": "1.5h"
  }
  return json.dumps(flight_info)

In [60]:
src_location = flight_response['loc_origin']
dest_location = flight_response['loc_destination']

print(src_location,dest_location)

Jodhpur Pune


In [61]:
chosen_function = response.choices[0].message.function_call.name
print(chosen_function)

get_flight_info


In [62]:
chosen_function = eval(chosen_function)

In [63]:
print(chosen_function)

<function get_flight_info at 0x7baf96273ac0>


In [64]:
chosen_function(src_location,dest_location)

'{"loc_origin": "Jodhpur", "loc_destination": "Pune", "flight_no": "A320", "itinerary_time": "1.5h"}'

In [65]:
flight = chosen_function(**flight_response)
print(flight)

{"loc_origin": "Jodhpur", "loc_destination": "Pune", "flight_no": "A320", "itinerary_time": "1.5h"}


In [70]:
function_name = response.choices[0].message.function_call.name
print(function_name)

get_flight_info


In [76]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages=[
        {"role":"user","content":prompt},
        {"role":"function","name":"get_flight_info","content":flight}
    ],
    functions = functions
)
print(response)

ChatCompletion(id='chatcmpl-9SlTrQEmwIZLzbwmAnKeE8gBfOsfp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The next flight from Jodhpur to Pune is on flight A320. The total time of the trip is 1.5 hours.', role='assistant', function_call=None, tool_calls=None))], created=1716642727, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=30, prompt_tokens=147, total_tokens=177))


In [83]:
from langchain import PromptTemplate, LLMChain
from langchain_openai import ChatOpenAI

# Create a PromptTemplate object
prompt = PromptTemplate(input_variables=[],template="Tell me a joke.")

# Create an LLMChain object
chain = LLMChain(prompt=prompt,llm=ChatOpenAI(model_name='gpt-3.5-turbo',verbose=True,openai_api_key=KEY,temperature=0.5),output_key="quiz",verbose=True)

# Call the invoke() method
output = chain.invoke({})

# Print the output
print(output)



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a joke.

> Finished chain.
{'quiz': 'Why did the scarecrow win an award? Because he was outstanding in his field!'}


Function Calling is basically connecting the Large Language Models to the external tools.